---

# Advanced `pandas`

In [ ]:
import pandas as pd

In [ ]:
loans = pd.read_csv("./data/loans.csv")

loans.head()

## Categorical data

Generally, text columns should be converted to `string` types.

However, if a column is categorical, we have a special data type we can use.

Why would we want to do that?

- the special `category` data type uses less memory
- we can also specify the order of the categories

Let's look at the first reason:

In [ ]:
loans["term"].memory_usage(deep=True)

Converting to `string` doesn't save any memory:

In [ ]:
loans["term"].astype("string").memory_usage(deep=True)

But converting it to a `category`:

In [ ]:
loans["term"].astype("category")

In [ ]:
loans["term"].astype("category").memory_usage(deep=True)

Let's take employment length as an example of an *ordered* categorical column:

In [ ]:
loans["emp_length"].value_counts().sort_index()

These aren't in the right order and sorting in ascending or descending order doesn't fix it.

In [ ]:
sorted(loans["emp_length"].dropna().unique())

In [ ]:
emp_dtype = pd.CategoricalDtype(['< 1 year', '1 year', '2 years',
                                 '3 years', '4 years', '5 years',
                                 '6 years', '7 years', '8 years',
                                 '9 years', '10+ years'], ordered=True)
loans["emp_length"].astype(emp_dtype)

What if we look at the `.value_counts()` now?

In [ ]:
loans["emp_length"].astype(emp_dtype).value_counts().sort_index()

## Reshaping

There are a few methods built in to `pandas` to help reshape your data.

Read all about them here: https://pandas.pydata.org/docs/user_guide/reshaping.html

One important one is to cross-reference categorical columns.

For example, what is the distribution of loan grades across different types of home owners?

In [ ]:
loans.pivot_table(index="home_ownership",
                  columns="grade",
                  values="id",
                  aggfunc="count"
                  )

`.crosstab()` does the same thing but has more limited options (but you could specify data from different sources since it's not tied to a single `DataFrame`). Different methods have different pros and cons!

In [ ]:
pd.crosstab(index=loans["home_ownership"],
            columns=loans["grade"])

Both methods allow you to specify what aggregation goes into the cells.

We could look at the average loan amount for different combinations of home ownership and grade:

In [ ]:
loans.pivot_table(index="home_ownership",
                  columns="grade",
                  values="loan_amnt",
                  aggfunc="mean"
                  )

Incidentally, this is where heatmaps come in handy

In [ ]:
import seaborn as sns

avg_by_home_and_grade = loans.pivot_table(index="home_ownership",
                                          columns="grade",
                                          values="loan_amnt",
                                          aggfunc="mean"
                                         )

_ = sns.heatmap(
    data=avg_by_home_and_grade,
    vmin=0,
    cmap="Greens",
    square=True
)

## Custom methods

Sometimes you want to perform a calculation on a column that's more advanced than what's already built in.

In these cases you can use the general `.apply` method, which applies a custom function to all rows.

Compared to other `pandas` operations, this is *slow*, but it's still quicker than using a loop.

Suppose we have rules for what makes a loan a "special" case:

- grade has to be B
- interest rate over 10%
- job title is "Accountant"

and we want to label our data with these rules.

Let's see the custom function approach.

When we write a custom function, the argument is either:

- a single value (if we want to apply the function to a single row or column)
- an entire row (or column) of data

In [ ]:
def is_special_loan(row):
    # only returns True if all conditions are met
    return row["grade"] == "B" and row["int_rate"] > 10 and row["emp_title"] == "Accountant"

We can then pass the function to `.apply` making sure `axis` is set to 1 (apply the function to each row), not 1 (which applies the function to each column)

In [ ]:
%%timeit

loans.apply(is_special_loan, axis=1)

Of course, we could have done this with "raw" `pandas` in a vectorised way:

In [ ]:
%%timeit

loans[(loans["grade"] == "B") & (loans["int_rate"] > 10) & (loans["emp_title"] == "Accountant")]

***Takeaway: only use `.apply` if there is no built-in `pandas` equivalent to what you want to do!***

## Method chaining

To really improve your `pandas` code, you can make use of **method chaining**.

This is when you add methods one after another, which you can do because `pandas` returns a copy of the `DataFrame` each time.

A classic read on the subject is here (a bit dated since `pandas` has much newer versions, but the ideas stand): https://tomaugspurger.net/posts/method-chaining/.

Consider this example:

In [ ]:
loans_1 = pd.read_csv("./data/loans.csv")

loans_1["installment_pct"] = loans_1["installment"] / loans_1["loan_amnt"]

# do we want to overwrite the raw loans data?
# or create loans_2?
loans_1 = loans_1.dropna(subset=["emp_length"])

loans_1["emp_length"] = loans_1["emp_length"].astype(emp_dtype)

income_by_emp = loans_1.groupby("emp_length")["annual_inc"].mean().sort_index(ascending=False)

_ = income_by_emp.plot(kind="barh")

You could also do all those operations in a single block:

- more readable
- steps are logically listed in the same place
- you don't overwrite the source data

In [ ]:
loans_1 = pd.read_csv("./data/loans.csv")

_ = (
    loans_1
    .assign(
        installment_pct=lambda df_: df_["installment"] / df_["loan_amnt"],
        emp_length=lambda df_: df_["emp_length"].astype(emp_dtype)
    )
    .dropna(subset=["emp_length"])
    .groupby("emp_length")
    ["annual_inc"]
    .mean()
    .sort_index(ascending=False)
    .plot(kind="barh")
)

This style of coding is entirely optional, but has a lot of benefits.

<h1 style="color: #fcd805">Exercise: Advanced pandas</h1>

Load in the Kickstarter data again.

For these questions, see if you can use the method chaining style of coding to solve the problems.

1. Convert the `state` column to be categorical. Choose a specific ordering for the states.

2. Calculate the average goal amount per state and display the results as a bar chart. Verify that the ordering you specified in your previous step has been taken into account.

3. What is the distribution of the `state` variable across different categories?

You should create a table with one row per category and one column per state. The cell values should be the number of projects for each category-state pair, e.g.

| |Failed|Succeed|
|---|---|---|
|**Art**|100|60|
|**Food**|72|103|
|**Music**|1412|835|

(Note, this isn't real data, just an illustration of the structure you're aiming for!)

4. Create a heatmap to visualise the table you created in step 3.

5. We're going to find the projects that *nearly* made it.

5.a. First, create a column to calculate the pledged amount as a percentage of the project goal

5.b. Now, write a Python function that determines is a project nearly made it.

The function should return True *only* if the following conditions are met for a single row:

- the goal was less than $100,000
- the state was `"failed"`
- the pledge as a percentage of the goal was over 90%

5.c. Use `.apply` on your `DataFrame` to call your function on all rows. Save the result in a column which indicates whether a project nearly made it.

5.d. Using your new column, how many projects nearly made it?

<h1 style="color: #fcd805">Exercise: Changing names</h1>

We're going to investigate how the popularity of names has changed over time.

What names are popular changes over time. To illustrate this, several articles pointed out that in 2013 only 28 babies were named Gary in the UK, whereas it was once a much more popular choice.

Let's see how the popularity of names has changed in recent years.

Our data comes from the Office of National Statistics: [baby names](https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/livebirths/datasets/babynamesenglandandwalesbabynamesstatisticsboys).

## Part 1 - reading and cleaning the data

There are 3 Excel files for 2019, 2020, and 2021.

1. Start by opening the 2019 file in Excel. Find where the raw count data is located for England and Wales (we want a list of boys names and their counts in a single, long table).

Now, use the `read_excel` method in `pandas` to read the data into a `pandas` DataFrame.

_Note: You will need to investigate and change some of the options!_

2. Drop any columns you don't need (`pandas` might have read in some empty data!)

3. Use the `rename` function to rename any columns that need better names.

4. Drop any rows you don't need. How can you identify rows that aren't actually part of your data?

5. Check the data types and convert any columns that need it.

6. Add a column to this DataFrame called `Year` which has the value 2019 for all rows.

7. Repeat the process for the 2020 and 2021 files.

You should end up with 3 DataFrames, one for each year.

Things to ensure:

- add the `Year` column with the correct year to each `DataFrame`
- column names and data types should be identical in all `DataFrames`

_Note: don't just run the same code with a different filename! Open each file and check that the data is stored in the same way, and amend your code if it isn't._

8. Use `pd.concat` to combine the 3 datasets into 1.

Call your combined DataFrame `names`.

9. Convert the name column to uppercase to ensure all names are uppercase

10. Drop any missing values.

11. Use your `names` `DataFrame` to see how many Garys were born in each year from 2019 to 2021.

## Part 2 - analysis

Now that we have our raw data, let's investigate it.

1. How many names are there in each year?

2. Which name had the most occurrences in a given year across the entire data? That is, what is the highest number of occurrences of a name in the dataset?

3. What is the most common first *letter* for boys' names?

4. What are the 5 most common names overall? Save these names into a separate variable (e.g. a list or a `Series`).

5. Now filter the `names` `DataFrame` so that only the top 5 names are kept (one row per name per year, so 15 rows in total)

6. We are going to reshape this `DataFrame` to plot the frequency of each of the 5 names over time.

To do this, our data needs to be **one column per line** on our line plot.

In this case, that means:

- one column per name
- one row per year
- values in the cells are the count of a particular name in a particular year

Something like this (with dummy data):

| |John|Joseph|David|
|---|---|---|---|
|**2019**|100|60|44|
|**2020**|72|103|230|
|**2021**|142|435|374|

Reshape the `DataFrame` to achieve this format.

7. Call the `.plot` function on this data. You should see years on the x-axis, count on the y-axis and one line per name.

Use your knowledge of `matplotlib` to clean up the visualisation and make it more presentation-ready, such as:

- adding a title and axis labels
- changing the "tick labels" (the values along the axis)

8. Now create a similar table, this time to track each name's rank over time.

The table should have:

- one row per name
- one column per year
- the values should be the **rank** of a name in a particular year (NaNs may be present for names that didn't appear in all years)

Something like this:

| |2019|2020|2021|
|---|---|---|---|
|**Robert**|407|392|232|
|**Stephen**|101|75|44|
|**Graham**|502|507|509|

9. Create a column in this table called `diff` which is the difference of a name's rank between 2021 and 2019.

10. Which names have moved up the most in the rankings and which ones have fallen the most?

_Note: remember, in rankings, the lower the better!_